Download additional dataset from Roboflow to merge with exisiting training dataset.
https://public.roboflow.com/object-detection/self-driving-car/

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Uf5F9mzBsL27ozIYiWvW")
project = rf.workspace("roboflow-gw7yv").project("self-driving-car")
version = project.version(3)
dataset = version.download("yolov8")

Manual copy to Masterfile folder

In [ ]:
import os

def adjust_label_indices(label_dir, adjustment):
    for label_file in os.listdir(label_dir):
        path = os.path.join(label_dir, label_file)
        with open(path, 'r+') as file:
            lines = file.readlines()
            file.seek(0)
            file.truncate()  # Clear existing content
            for line in lines:
                parts = line.split()
                parts[0] = str(int(parts[0]) + adjustment)  # Adjust the class index
                file.write(" ".join(parts) + "\n")

# Adjust labels for the self-driving car dataset (class indices start at 7)
adjust_label_indices('/home/mangguai/capstone/code/Master_Dataset/export/labels', 7)

In [ ]:
import os
from collections import defaultdict

def count_classes(label_dir):
    class_counts = defaultdict(int)  # Dictionary to store counts of each class index
    
    for label_file in os.listdir(label_dir):
        path = os.path.join(label_dir, label_file)
        with open(path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                class_index = line.split()[0]  # Get the class index from each line
                class_counts[int(class_index)] += 1  # Increment count for this class

    return class_counts

# Example usage
label_dir = '/home/mangguai/capstone/code/Master_Dataset/export/labels'
class_counts = count_classes(label_dir)
print("Class counts:")
for class_index in sorted(class_counts):
    print(f"Class {class_index}: {class_counts[class_index]} images")
print("Number of unique classes:", len(class_counts))


In [ ]:
import os
from collections import defaultdict

def find_label_files_with_classes(label_dir, class_range):
    class_files = defaultdict(list)  # Dictionary to store lists of files for each class index

    for label_file in os.listdir(label_dir):
        path = os.path.join(label_dir, label_file)
        with open(path, 'r') as file:
            lines = file.readlines()
            found_classes = set()  # To avoid adding the same file multiple times for the same class
            for line in lines:
                class_index = int(line.split()[0])  # Get the class index from each line
                if class_index in class_range and class_index not in found_classes:
                    if len(class_files[class_index]) < 5:  # Ensure no more than 5 files are stored
                        class_files[class_index].append(label_file)
                    found_classes.add(class_index)

    return class_files

# Define the class range you are interested in
class_range = set(range(7, 18))  # From 7 to 17

# Example usage
label_dir = '/home/mangguai/capstone/code/Master_Dataset/export/labels'
class_files = find_label_files_with_classes(label_dir, class_range)

# Printing the results with newline for each file name
for class_index in class_range:
    if class_index in class_files:  # Check if there are any files for the class
        files = "\n".join(class_files[class_index])  # Join files with newline character
        print(f"Class {class_index} has the following label files:\n{files}\n")
    else:
        print(f"Class {class_index} has no label files.\n")



Based on your description, you want to merge specific traffic light classes with their corresponding "left" direction classes and then reassign the indices to reduce the total number of classes. Your mapping appears mostly correct according to your specifications, but let's review and confirm the reassignments for clarity:

Biker (7) remains 7.
Car (8) remains 8.
Pedestrians (9) remains 9.
Traffic Light (10) remains 10.
Traffic Light-Green (11) and Traffic Light-GreenLeft (12) both become 11.
Traffic Light-Red (13) and Traffic Light-RedLeft (14) both become 12.
Traffic Light-Yellow (15) and Traffic Light-YellowLeft (16) both become 13.
Truck (17) becomes 14.
The assignment seems correct based on the pattern you are following: merging the standard and "left" variations of traffic lights into a single class and subsequently renumbering the remaining classes.

Here's the simplified class structure based on your description:

7: Biker
8: Car
9: Pedestrians
10: Traffic Light
11: Traffic Light-Green (merged with Traffic Light-GreenLeft)
12: Traffic Light-Red (merged with Traffic Light-RedLeft)
13: Traffic Light-Yellow (merged with Traffic Light-YellowLeft)
14: Truck


In [ ]:
import os

def adjust_classes(label_dir, mapping):
    for label_file in os.listdir(label_dir):
        path = os.path.join(label_dir, label_file)
        with open(path, 'r+') as file:
            lines = file.readlines()
            file.seek(0)
            file.truncate()  # Clear the file
            for line in lines:
                parts = line.split()
                class_index = int(parts[0])
                if class_index in mapping:
                    parts[0] = str(mapping[class_index])  # Update class index based on mapping
                file.write(" ".join(parts) + "\n")

# Define your class mapping
mapping = {
    7: 7,
    8: 8,
    9: 9,
    10: 10,
    11: 11,
    12: 11,
    13: 12,
    14: 12,
    15: 13,
    16: 13,
    17: 14
}

# Example usage
label_dir = '/home/mangguai/capstone/code/Master_Dataset/export/labels'
adjust_classes(label_dir, mapping)


recheck the class total numbers


In [ ]:
import os
from collections import defaultdict

def count_classes(label_dir):
    class_counts = defaultdict(int)  # Dictionary to store counts of each class index
    
    for label_file in os.listdir(label_dir):
        path = os.path.join(label_dir, label_file)
        with open(path, 'r') as file:
            lines = file.readlines()
            for line in lines:
                class_index = line.split()[0]  # Get the class index from each line
                class_counts[int(class_index)] += 1  # Increment count for this class

    return class_counts

# Example usage
label_dir = '/home/mangguai/capstone/code/Master_Dataset/export/labels'
class_counts = count_classes(label_dir)
print("Class counts:")
for class_index in sorted(class_counts):
    print(f"Class {class_index}: {class_counts[class_index]} images")
print("Number of unique classes:", len(class_counts))


The data now ready to merge. need to edit yaml file

now the image and label ready to split

can you help me split into test/train/valid?
can access to label file to see the total number of different class, help me split 70% for training, 20% for validation and 10% for testing...
understand that some image have multiple labels, please help me prioritize the lowest count first to split with that

the file location as below:
File to split out from: /home/mangguai/capstone/code/Master_Dataset/export (inside there is images and labels folder)
Split to: /home/mangguai/capstone/code/Master_Dataset/test (inside there is images and labels folder)
/home/mangguai/capstone/code/Master_Dataset/train (inside there is images and labels folder)
/home/mangguai/capstone/code/Master_Dataset/valid (inside there is images and labels folder)

reminder: please split both image and label file with the same name under same test/train/valid folder


To split the dataset into training, validation, and testing sets while considering class frequencies to ensure a balanced split, you can use a Python script. This process involves counting occurrences of each class in the labels, prioritizing splits based on class rarity, and then distributing images and their corresponding labels into the designated directories. Here's a step-by-step Python script to help you achieve this:

Count class frequencies in the labels.
Assign images to splits based on class rarity, aiming for a distribution of 70% training, 20% validation, and 10% testing.
Move files to their respective directories.

Key Points of the Script:
Counting Classes and Images: It calculates how often each class appears and which classes each image contains.
Sorting by Rarity: Images are sorted based on the rarity of the classes they contain. This helps in distributing less frequent classes more evenly across the splits.
Splitting the Data: Uses train_test_split from sklearn to split the dataset into the desired ratios.
Moving Files: Corresponding image and label files are moved to their new directories.
Considerations:
File Types: This script assumes label files are .txt and image files are .jpg. Adjust the extensions if your dataset uses different formats.
Random Seed: A random seed (42) is used for reproducibility in splits.
Directory Structure: Ensure that the directories exist before running the script or add code to create them if they do not.

In [ ]:
import os
import shutil
from collections import defaultdict
from sklearn.model_selection import train_test_split

def count_classes(label_dir):
    class_counts = defaultdict(int)
    image_classes = defaultdict(set)
    
    for label_file in os.listdir(label_dir):
        path = os.path.join(label_dir, label_file)
        with open(path, 'r') as file:
            for line in file:
                class_index = line.split()[0]
                class_counts[int(class_index)] += 1
                image_classes[label_file].add(int(class_index))
    return class_counts, image_classes

def split_datasets(base_dir, output_dirs, split_ratio=(0.7, 0.2, 0.1)):
    label_dir = os.path.join(base_dir, 'labels')
    image_dir = os.path.join(base_dir, 'images')
    
    class_counts, image_classes = count_classes(label_dir)
    
    # Sort images by the rarity of their classes
    sorted_images = sorted(image_classes.keys(), key=lambda x: min(class_counts[i] for i in image_classes[x]))
    
    # Split images into training, validation, and testing
    train_val, test = train_test_split(sorted_images, test_size=split_ratio[2], random_state=42)
    train, val = train_test_split(train_val, test_size=split_ratio[1] / (split_ratio[0] + split_ratio[1]), random_state=42)
    
    # Function to move files
    def move_files(files, dest_dir):
        for file_name in files:
            # Move label file
            shutil.move(os.path.join(label_dir, file_name), os.path.join(dest_dir, 'labels', file_name))
            # Move corresponding image file
            image_file_name = file_name.replace('.txt', '.jpg')  # Assuming image files are .jpg
            shutil.move(os.path.join(image_dir, image_file_name), os.path.join(dest_dir, 'images', image_file_name))
    
    # Move files to respective directories
    move_files(train, os.path.join(output_dirs['train']))
    move_files(val, os.path.join(output_dirs['valid']))
    move_files(test, os.path.join(output_dirs['test']))

# Example usage
base_dir = '/home/mangguai/capstone/code/Master_Dataset/export'
output_dirs = {
    'train': '/home/mangguai/capstone/code/Master_Dataset/train',
    'valid': '/home/mangguai/capstone/code/Master_Dataset/valid',
    'test': '/home/mangguai/capstone/code/Master_Dataset/test'
}
split_datasets(base_dir, output_dirs)


there are 3500 images leftover due to no labelling
manual update yaml file

Now the dataset are ready to be train.

In [23]:
import os

# Define the classes and their corresponding indices
class_names = [
    'curveleft_sign',           # Class 0
    'curveright_sign',          # Class 1
    'mandatorystop_sign',       # Class 2
    'noentry_sign',             # Class 3
    'nojaywalking_sign',        # Class 4
    'pedestriancrossing_sign',  # Class 5
    'zebracrossing_sign',       # Class 6
    'biker',                    # Class 7
    'car',                      # Class 8
    'pedestrian',               # Class 9
    'trafficlight',             # Class 10
    'trafficlight_green',       # Class 11
    'trafficlight_red',         # Class 12
    'trafficlight_yellow',      # Class 13
    'truck'                     # Class 14
]
class_indices = {name: index for index, name in enumerate(class_names)}

# Image dimensions
img_width, img_height = 640, 640

# Path to the folder containing the annotation files
folder_path = 'Master_Dataset/train/labels'
output_folder_path = 'Master_Dataset/train/modified_labels'  # New output folder

os.makedirs(output_folder_path, exist_ok=True)  # Ensure the output directory exists

# Function to normalize coordinates
def normalize_coordinates(x, y, w, h, img_width, img_height):
    dw = 1.0 / img_width
    dh = 1.0 / img_height
    x_center = (x + w / 2.0) * dw
    y_center = (y + h / 2.0) * dh
    width = w * dw
    height = h * dh
    return x_center, y_center, width, height

# Process each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".txt"):  # assuming the annotation files are .txt files
        file_path = os.path.join(folder_path, filename)
        output_file_path = os.path.join(output_folder_path, filename)  # Output file path
        with open(file_path, 'r') as file:
            lines = file.readlines()

        new_lines = []
        for line in lines:
            parts = line.strip().split()
            if len(parts) == 5:  # Ensure this is a YOLO format line
                class_index, x, y, w, h = map(float, parts)
                # Normalize coordinates
                x_center, y_center, width, height = normalize_coordinates(x, y, w, h, img_width, img_height)
                new_line = f"{int(class_index)} {x_center} {y_center} {width} {height}"
                new_lines.append(new_line)

        # Write the converted lines to a new file in the output directory
        with open(output_file_path, 'w') as file:
            file.write('\n'.join(new_lines))


In [2]:
import subprocess

# Command to run
command = "yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Master_Dataset/data.yaml epochs=20 imgsz=640 batch=8"

# Execute the command
subprocess.run(command, shell=True)

#134min for 20 epochs


New https://pypi.org/project/ultralytics/8.2.10 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/mangguai/capstone/code/Master_Dataset/data.yaml, epochs=20, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_wid

CompletedProcess(args='yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Master_Dataset/data.yaml epochs=20 imgsz=640 batch=8', returncode=0)

In [24]:
import subprocess

# Command to run
command = "yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Master_Dataset/data.yaml epochs=25 imgsz=640 batch=8"

# Execute the command
subprocess.run(command, shell=True)

#165min for 25 epochs

New https://pypi.org/project/ultralytics/8.2.10 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/home/mangguai/capstone/code/Master_Dataset/data.yaml, epochs=25, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_wid

CompletedProcess(args='yolo train model=yolov8s.pt data=/home/mangguai/capstone/code/Master_Dataset/data.yaml epochs=25 imgsz=640 batch=8', returncode=0)

In [40]:
#before running this command, make sure you are in the correct directory, check with pwd
!yolo task=detect mode=predict model=/home/mangguai/capstone/code/runs/detect/train5/weights/best.pt conf=0.25 source=/home/mangguai/capstone/data/greenlight.jpg


Ultralytics YOLOv8.0.196 🚀 Python-3.11.8 torch-2.2.2 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)
Model summary (fused): 168 layers, 11131389 parameters, 0 gradients, 28.5 GFLOPs

image 1/1 /home/mangguai/capstone/data/greenlight.jpg: 416x640 1 biker, 2 pedestrians, 92.6ms
Speed: 3.1ms preprocess, 92.6ms inference, 99.3ms postprocess per image at shape (1, 3, 416, 640)
Results saved to runs/detect/predict9
💡 Learn more at https://docs.ultralytics.com/modes/predict
